In [ ]:
import cv2
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.display import clear_output
import ipywidgets as widgets

# ─── CONFIG ─────────────────────────────────────────
BASE_DATE = "16_7_2025"
BASE_PATH = Path(f"/Users/at/Orbit_Red_Blob/Data/UKE_Plot_14_DD_{BASE_DATE}/sample")
IMAGE_NAME = "DJI_0382_zoomed_27.jpg"
img_bgr = cv2.imread(str(BASE_PATH / IMAGE_NAME))
if img_bgr is None:
    raise FileNotFoundError(f"Couldn’t load image: {BASE_PATH/IMAGE_NAME}")
img_hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)

# ─── WIDGETS ───────────────────────────────────────
# Helper to make a slider
def make_slider(name, minv, maxv, init):
    return widgets.IntSlider(
        value=init, min=minv, max=maxv, step=1, description=name, continuous_update=False
    )

# Create sliders for both red ranges
h1_lo = make_slider("H1 low",   0, 179,  0)
h1_hi = make_slider("H1 high",  0, 179, 10)
s1_lo = make_slider("S1 low",   0, 255,100)
s1_hi = make_slider("S1 high",  0, 255,255)
v1_lo = make_slider("V1 low",   0, 255, 20)
v1_hi = make_slider("V1 high",  0, 255,150)

h2_lo = make_slider("H2 low",   0, 179,160)
h2_hi = make_slider("H2 high",  0, 179,179)
s2_lo = make_slider("S2 low",   0, 255,100)
s2_hi = make_slider("S2 high",  0, 255,255)
v2_lo = make_slider("V2 low",   0, 255, 25)
v2_hi = make_slider("V2 high",  0, 255,188)

save_btn = widgets.Button(description="Save Output")
out = widgets.Output()

# ─── PROCESSING & DISPLAY ──────────────────────────
def update_display(_=None):
    # build masks
    lower1 = np.array([h1_lo.value, s1_lo.value, v1_lo.value])
    upper1 = np.array([h1_hi.value, s1_hi.value, v1_hi.value])
    lower2 = np.array([h2_lo.value, s2_lo.value, v2_lo.value])
    upper2 = np.array([h2_hi.value, s2_hi.value, v2_hi.value])

    m1 = cv2.inRange(img_hsv, lower1, upper1)
    m2 = cv2.inRange(img_hsv, lower2, upper2)
    red_mask = cv2.bitwise_or(m1, m2)
    inv = cv2.bitwise_not(red_mask)

    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    gray_bgr = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    red_part  = cv2.bitwise_and(img_bgr, img_bgr, mask=red_mask)
    gray_part = cv2.bitwise_and(gray_bgr, gray_bgr, mask=inv)
    combo = cv2.add(red_part, gray_part)

    # show inline
    with out:
        clear_output(wait=True)
        plt.figure(figsize=(8,6))
        plt.imshow(cv2.cvtColor(combo, cv2.COLOR_BGR2RGB))
        plt.axis("off")
    # store for saving
    update_display.last = combo

# call on any slider change
for w in (h1_lo,h1_hi,s1_lo,s1_hi,v1_lo,v1_hi,
          h2_lo,h2_hi,s2_lo,s2_hi,v2_lo,v2_hi):
    w.observe(update_display, names='value')

# save handler
def on_save(_):
    save_path = BASE_PATH / "output_tuned.png"
    cv2.imwrite(str(save_path), update_display.last)
    with out:
        print(f"Saved to {save_path}")

save_btn.on_click(on_save)

# ─── LAYOUT ────────────────────────────────────────
controls = widgets.VBox([
    widgets.HBox([h1_lo, h1_hi, s1_lo, s1_hi, v1_lo, v1_hi]),
    widgets.HBox([h2_lo, h2_hi, s2_lo, s2_hi, v2_lo, v2_hi]),
    save_btn
])

display(controls, out)

# initial draw
update_display()
